In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import sys
import os
from datetime import datetime
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==========================================
# 0. 설정 (Configuration)
# ==========================================
# 데이터 파일 경로
DATA_PATH = "data/final_features.csv"

# 학습할 포지션 목록 (순차적으로 실행됨)
TARGET_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE', 'BOT_DUO']

# [사용자 요청 반영] 전체 22개 중 'level', 'levelDiff' 제외한 20개 Feature
FEATURE_COLS = [
    # 성장
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    # 전투
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills',
    # 운영 및 기타
    'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    # 격차 (Diff)
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    # 비율
    'KDA', 'killParticipation', 'damageShare'
]
# 제외됨: 'level', 'levelDiff'

TARGET_COL = 'totalGold'

# ==========================================
# 1. 로깅 클래스 & 환경 설정
# ==========================================
class DualLogger(object):
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "a", encoding='utf-8')
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
    def flush(self):
        self.terminal.flush()
        self.log.flush()

# 폴더 생성
os.makedirs('logs', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('output', exist_ok=True)

# 로그 파일 설정
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"logs/train_all_positions_{timestamp}.txt"
sys.stdout = DualLogger(log_filename)

# 시각화 폰트 설정 (한글 깨짐 방지)
import platform
system_name = platform.system()
if system_name == 'Darwin': plt.rcParams['font.family'] = 'AppleGothic'
elif system_name == 'Windows': plt.rcParams['font.family'] = 'Malgun Gothic'
else: plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# ==========================================
# 2. 데이터 로드 (한 번만 실행)
# ==========================================
def load_data():
    print(f"Loading data from {DATA_PATH}...")
    if not os.path.exists(DATA_PATH):
        print(f"Error: 데이터 파일을 찾을 수 없습니다. ({DATA_PATH})")
        sys.exit(1)

    df = pd.read_csv(DATA_PATH)

    # 0분 데이터 제거
    df = df[df['minute'] > 0].reset_index(drop=True)

    # 결측치 처리 (NaN -> 0, 특히 soloKills 등)
    df.fillna(0, inplace=True)

    print(f"✅ Data Loaded. Total Rows: {len(df)}")
    return df

# ==========================================
# 3. 포지션별 학습 함수
# ==========================================
def train_for_position(df, position):
    print("\n" + "="*60)
    print(f"🚀 Starting Training for Position: {position}")
    print("="*60)

    # 1. 포지션 필터링
    pos_df = df[df['position'] == position].reset_index(drop=True)

    if len(pos_df) == 0:
        print(f"⚠️ Warning: '{position}' 데이터가 없습니다. 스킵합니다.")
        return

    print(f"📊 Rows for {position}: {len(pos_df)}")

    # Feature 확인
    X = pos_df[FEATURE_COLS]
    y = pos_df[TARGET_COL]
    groups = pos_df['matchId']

    # 2. Train/Test 분할 (GroupShuffleSplit)
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, test_idx = next(gss.split(X, y, groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_test = groups.iloc[test_idx]

    # 3. 모델 학습
    # [수정됨] early_stopping_rounds를 여기(생성자)로 이동했습니다.
    model = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        n_jobs=-1,
        random_state=42,
        tree_method='hist',
        early_stopping_rounds=50  # 👈 여기에 추가!
    )

    print(f"Training XGBoost for {position}...")

    # [수정됨] fit() 안에서는 early_stopping_rounds 제거
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=True
    )

    # 4. 평가
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"\n✅ [{position}] Evaluation:")
    print(f"   MAE : {mae:.2f} Gold")
    print(f"   RMSE: {rmse:.2f} Gold")
    print(f"   R2  : {r2:.4f}")

    # 5. 저장 (모델 & 결과)
    # 모델 저장
    model_filename = f"models/xgb_gold_predictor_{position}_{timestamp}.pkl"
    joblib.dump(model, model_filename)
    print(f"   💾 Model Saved: {model_filename}")

    # 예측 결과 CSV 저장
    results_df = X_test.copy()
    results_df['actualGold'] = y_test
    results_df['predictedGold'] = y_pred
    results_df['error'] = results_df['predictedGold'] - results_df['actualGold']
    results_df['matchId'] = groups_test
    results_csv = f'output/test_predictions_{position}_{timestamp}.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"   📄 Predictions Saved: {results_csv}")

    # 6. Feature Importance 시각화
    plt.figure(figsize=(12, 10))
    xgb.plot_importance(model, max_num_features=20, importance_type='gain', height=0.6, grid=False)
    plt.title(f'Feature Importance (Gain) - {position}')
    plt.tight_layout()
    plt.savefig(f'output/feature_importance_{position}_{timestamp}.png')
    plt.close()
    print(f"   📊 Plot Saved: output/feature_importance_{position}_{timestamp}.png")

# ==========================================
# 4. 메인 실행
# ==========================================
if __name__ == "__main__":
    # 데이터 로드
    full_df = load_data()

    # 순차적으로 모든 포지션 학습
    for pos in TARGET_POSITIONS:
        train_for_position(full_df, pos)

    print("\n" + "="*60)
    print("🎉 All Positions Trained Successfully!")
    print(f"📂 Check 'output/' and 'models/' directories.")
    print("="*60)

<Figure size 1200x1000 with 0 Axes>

<Figure size 1200x1000 with 0 Axes>

<Figure size 1200x1000 with 0 Axes>

<Figure size 1200x1000 with 0 Axes>